In [1]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
from PIL import Image, ImageOps

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = str(self.labels_df.iloc[idx, 0])
        try:
            img_path = os.path.join(self.image_dir, img_name + '.jpg')  
            image = Image.open(img_path).convert("RGB")
        except:
            img_path = os.path.join(self.image_dir, img_name + '.png')  
            image = Image.open(img_path).convert("RGB")
        
        label = int(self.labels_df.iloc[idx, 2])  # warna kolom index 2, jenis kolom index 1

        if self.transform:
            image = self.transform(image)

        return image, label

csv_file = '/kaggle/input/penyisihan-hology-7-data-mining-competition/train.csv'
train_dir = '/kaggle/input/segmented-plis/train_segmented'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

# Create the dataset
train_dataset = CustomDataset(csv_file=csv_file, image_dir=train_dir, transform=transform)

# DataLoader for batching
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImageCNN(nn.Module):
    def __init__(self, num_classes=5):  
        super(ImageCNN, self).__init__()
        
        # Convolutional layers with ReLU activations
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 32 * 32, 512)  # Adjusted input size based on 256x256 image
        self.fc2 = nn.Linear(512, num_classes)  # Output layer for the number of classes

    def forward(self, x):
        # First conv layer + ReLU + pooling
        x = self.pool(F.relu(self.conv1(x)))        
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = torch.flatten(x, start_dim=1)  
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        
        # Output layer (logits)
        x = self.fc2(x)
        
        return x

# Example usage:
num_classes = 5  # Set the correct number of classes for your dataset
model = ImageCNN(num_classes=num_classes)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
import torch.nn.functional as F

def train_model(model, train_loader, criterion, optimizer, num_epochs=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)  
    
    best_accuracy = 0.0
    lowest_loss = float('inf')
    best_model_state = None

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        total = 0


        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to device

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Accumulate the loss
            running_loss += loss.item()

            # Compute predictions and accuracy
            _, predicted = torch.max(outputs.data, 1)  # Get the class with highest score
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calculate accuracy
        accuracy = 100 * correct / total
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = (correct / total) * 100
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.6f}, Accuracy: {accuracy:.2f}%")

        # Check if this is the best model
        if epoch_acc > best_accuracy or epoch_loss < lowest_loss:
            best_accuracy = epoch_acc
            lowest_loss = epoch_loss
            best_model_state = model.state_dict()  # Save the model's state

    # Save the best model after training
    if best_model_state is not None:
        torch.save(best_model_state, 'best_model.pth')
        print(f"Best Model Saved with Accuracy: {best_accuracy:.2f}%, Loss: {lowest_loss:.4f}")

train_model(model, train_loader, criterion, optimizer)

Epoch [1/50], Loss: 1.080767, Accuracy: 85.46%
Epoch [2/50], Loss: 0.170478, Accuracy: 94.85%
Epoch [3/50], Loss: 0.063063, Accuracy: 97.94%
Epoch [4/50], Loss: 0.058433, Accuracy: 98.46%
Epoch [5/50], Loss: 0.023735, Accuracy: 99.36%
Epoch [6/50], Loss: 0.007135, Accuracy: 99.74%
Epoch [7/50], Loss: 0.051919, Accuracy: 98.46%
Epoch [8/50], Loss: 0.170477, Accuracy: 97.30%
Epoch [9/50], Loss: 0.065666, Accuracy: 98.58%
Epoch [10/50], Loss: 0.033036, Accuracy: 99.36%
Epoch [11/50], Loss: 0.283197, Accuracy: 96.40%
Epoch [12/50], Loss: 0.137230, Accuracy: 97.17%
Epoch [13/50], Loss: 0.112488, Accuracy: 98.33%
Epoch [14/50], Loss: 0.127059, Accuracy: 97.68%
Epoch [15/50], Loss: 0.024609, Accuracy: 98.97%
Epoch [16/50], Loss: 0.002314, Accuracy: 99.87%


In [5]:
import pandas as pd
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

# Custom Dataset class for test data (no labels)
class TestDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with image IDs.
            image_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.image_df = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_df)

    def __getitem__(self, idx):
        img_name = str(self.image_df.iloc[idx, 0])  # Convert the image ID to string
        try:
            img_path = os.path.join(self.image_dir, img_name + '.jpg')  # Adjust extension if needed
            image = Image.open(img_path).convert("RGB")
        except:
            img_path = os.path.join(self.image_dir, img_name + '.png')  # Adjust extension if needed
            image = Image.open(img_path).convert("RGB")
            
        if self.transform:
            image = self.transform(image)

        return image, img_name

# Example usage for test data
csv_file = '/kaggle/input/penyisihan-hology-7-data-mining-competition/sample_submission.csv'
# test_dir = '/kaggle/input/penyisihan-hology-7-data-mining-competition/test/test'
test_dir = '/kaggle/input/plis-segmented/test_segmented'

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the test dataset
test_dataset = TestDataset(csv_file=csv_file, image_dir=test_dir, transform=transform)

# DataLoader for batching
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [6]:
# model = torch.load('/kaggle/input/warna_332/pytorch/default/1/warna_330.pth')

In [7]:
def predict(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()  # Set model to evaluation mode

    predictions = []
    
    with torch.no_grad():  # No need to calculate gradients
        for images, img_ids in test_loader:
            images = images.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # Get the predicted class

            # Collect results
            for img_id, pred in zip(img_ids, predicted):
                predictions.append((img_id, pred.item()))  # Store image ID and prediction

    return predictions

test_predictions = predict(model, test_loader)

In [8]:
import csv

def save_predictions_to_csv(predictions, output_file):
    # Save the predictions (image_id, predicted_class) to a CSV file
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["id", "warna"])  # Header
        writer.writerows(predictions)

# Example usage to save predictions to a file
output_file = '330.csv'
save_predictions_to_csv(test_predictions, output_file)